In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split

from tensorflow.keras.callbacks import LearningRateScheduler
from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv2D, Flatten, BatchNormalization
from keras.callbacks import ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

        
import warnings
warnings.filterwarnings("ignore")

In [ ]:
base_train = pd.read_csv('/kaggle/input/digit-recognizer/train.csv')
base_test = pd.read_csv('/kaggle/input/digit-recognizer/test.csv')
print(base_train.shape, base_test.shape)

In [ ]:
base_train.head()

# Total Amount of Unique Labels in the Database

In [ ]:
# Lista ordenada dos Rótulos da classe
sorted(base_train["label"].unique())

# Separating the Pixels of the Images from the Labels

In [ ]:
x = base_train.drop("label", axis = 1)
y = base_train["label"]

In [ ]:
print(x.shape, y.shape)

# Separating Training Database and Test Database

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.3, random_state = 1)

In [ ]:
print(x_train.shape, x_test.shape)
print(y_train.shape, y_test.shape)

# Reshape 

In [ ]:
x_train = np.array(x_train).astype("float64")
x_test = np.array(x_test).astype("float64")

x_train = x_train.reshape(-1, 28, 28, 1)
x_test = x_test.reshape(-1, 28, 28, 1)

In [ ]:
x_train.shape

In [ ]:
x_train[1].dtype

In [ ]:
fig = plt.figure(figsize = (13, 8))
for i in range(9):
    plt.subplot(330 + 1 + i)
    plt.imshow(x_train[i])
plt.show()

# Normalize 

In [ ]:
x_train = x_train / 255.0
x_test = x_test / 255.0

In [ ]:
x_train.shape[1:]

# Structure of the Convolutional Neural Network with Tensorflow and Keras

In [ ]:
classifier = Sequential()

classifier.add(Conv2D(64, kernel_size = 3, activation = "relu", padding="same", input_shape = (28, 28, 1)))
classifier.add(BatchNormalization())

classifier.add(Conv2D(64, kernel_size = 3, activation = "relu"))
classifier.add(BatchNormalization())

classifier.add(Conv2D(64, kernel_size = 5, strides=2, activation = "relu", padding="same"))
classifier.add(BatchNormalization())
classifier.add(Dropout(0.4))

classifier.add(Conv2D(64, kernel_size = 3, activation = "relu"))
classifier.add(BatchNormalization())

classifier.add(Conv2D(64, kernel_size = 3, activation = "relu"))
classifier.add(BatchNormalization())

classifier.add(Conv2D(64, kernel_size = 5, strides=2, activation = "relu", padding="same"))
classifier.add(BatchNormalization())
classifier.add(Dropout(0.4))

classifier.add(Conv2D(64, kernel_size = 4, activation = "relu"))
classifier.add(BatchNormalization())

classifier.add(Flatten())
classifier.add(Dropout(0.3))

classifier.add(Dense(units = 10, activation = "softmax"))

In [ ]:
classifier.summary()

# Convolutional Neural Network Architecture

In [ ]:
from keras.utils import plot_model
plot_model(classifier, to_file='classifier.png', show_shapes=True, show_layer_names=True)
from IPython.display import Image
Image("classifier.png")

# Compiling the Model

In [ ]:
classifier.compile(optimizer = "adam", loss = "sparse_categorical_crossentropy", metrics = ["accuracy"])

# Training the Model

In [ ]:
learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', 
                                            patience=2, 
                                            verbose=2, 
                                            factor=0.5, 
                                            min_lr=0.00001)

In [ ]:
annealer = LearningRateScheduler(lambda x: 1e-3 * 0.95 ** x)

# Image Data Generator

In [ ]:
data_generator = ImageDataGenerator(
        featurewise_center=False,
        samplewise_center=False,
        featurewise_std_normalization=False,
        samplewise_std_normalization=False,
        zca_whitening=False,
        rotation_range=10,
        zoom_range = 0.1,
        width_shift_range=0.2,
        height_shift_range=0.2,
        horizontal_flip=False,
        vertical_flip=False)

data_generator.fit(x_train)

In [ ]:
history = classifier.fit(data_generator.flow(x_train,y_train, batch_size=64),
                         epochs = 50, validation_data = (x_test,y_test) ,
                         callbacks=[learning_rate_reduction, annealer],
                         verbose = 2, steps_per_epoch=x_train.shape[0] // 64)

In [ ]:
 #history.history 

# Evaluation of the Trained Model

In [ ]:

def graphic_evaluation(history,materics,graphic_title):
    plt.plot(history.history[materics])
    plt.plot(history.history['val_'+materics])
    plt.title(graphic_title, fontsize = 16)
    plt.legend(['train','test'])
    plt.xlabel('Epochs', fontsize = 15)
    plt.ylabel(materics, fontsize = 15)
    plt.show()

In [ ]:
# ploting the accuracy
graphic_evaluation(history,'accuracy', 'Model Accuracy')

In [ ]:
# ploting the model loss
graphic_evaluation(history, 'loss', 'Model Loss')

# Save submition

In [ ]:
x_test_sub = base_test.copy()

In [ ]:
x_test_sub = x_test_sub.values.reshape(-1,28,28,1)
x_test_sub = x_test_sub.astype('float32')/255.0

In [ ]:
pred_x_test_sub = classifier.predict(x_test_sub)

In [ ]:
pred_x_test_sub

In [ ]:
my_submition = pd.DataFrame({'ImageId':base_test.index.values + 1, 'Label':np.argmax(pred_x_test_sub, axis=1)})

In [ ]:
my_submition

In [ ]:
my_submition.to_csv('digit_mnist.csv', index=False)

In [ ]:
ls